# 01__DESeq2

In [1]:
source("https://bioconductor.org/biocLite.R")
biocLite("DESeq2")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Bioconductor version 3.7 (BiocInstaller 1.30.0), ?biocLite for help
A newer version of Bioconductor is available for this version of R,
  ?BiocUpgrade for help
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.7 (BiocInstaller 1.30.0), R 3.5.1 (2018-07-02).
Installing package(s) ‘DESeq2’
also installing the dependencies ‘bit’, ‘bit64’, ‘blob’, ‘memoise’, ‘RSQLite’, ‘XML’, ‘checkmate’, ‘AnnotationDbi’, ‘annotate’, ‘Formula’, ‘latticeExtra’, ‘acepack’, ‘gridExtra’, ‘htmlTable’, ‘viridis’, ‘genefilter’, ‘locfit’, ‘geneplotter’, ‘Hmisc’, ‘RcppArmadillo’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'assertthat', 'backports', 'BH', 'boot', 'broom', 'callr',
  'caret', 'class', 'cli', 'clipr', 'cluster', 'codetools', 'colorspace',
  'curl', 'data.table', 'dbplyr', 'ddalpha', 'digest', 'dimRed', 'dplyr',
  'evaluate', 'fansi', 'forcats', 'formatR', 'geometry', 

In [2]:
library("DESeq2")

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which

## 1. import data

In [35]:
human_cts <- read.csv("../../../data/03__rna_seq/02__for_DESeq2/hESC_all.counts.txt", sep="\t", row.names="gene_id")
human_cts <- as.matrix(human_cts)
head(human_cts)

rep1 rep2
ENSG00000223972  2    0  
ENSG00000227232 45   36  
ENSG00000243485  3    0  
ENSG00000237613  0    0  
ENSG00000268020  0    0  
ENSG00000240361  0    0

In [36]:
mouse_cts <- read.csv("../../../data/03__rna_seq/02__for_DESeq2/mESC_all.counts.txt", sep="\t", row.names="gene_id")
mouse_cts <- as.matrix(mouse_cts)
head(mouse_cts)

rep1 rep2 rep3
ENSMUSG00000102693 0    0    0   
ENSMUSG00000064842 0    0    0   
ENSMUSG00000051951 2    2    2   
ENSMUSG00000102851 0    0    0   
ENSMUSG00000103377 0    0    0   
ENSMUSG00000104017 0    0    0

In [37]:
human_cols <- read.csv("../../../data/03__rna_seq/02__for_DESeq2/hESC_all.cols.txt", sep="\t", row.names="column")
head(human_cols)

condition
rep1 rep1     
rep2 rep2

In [38]:
mouse_cols <- read.csv("../../../data/03__rna_seq/02__for_DESeq2/mESC_all.cols.txt", sep="\t", row.names="column")
head(mouse_cols)

condition
rep1 rep1     
rep2 rep2     
rep3 rep3

In [17]:
orth_cts <- read.csv("../../../data/03__rna_seq/02__for_DESeq2/orths.counts.txt", sep="\t", row.names="orth_id")
orth_cts <- as.matrix(orth_cts)
head(orth_cts)

hESC_rep1 hESC_rep2 mESC_rep1 mESC_rep2
ENSG00000187634__ENSMUSG00000096351    91        67         5         5    
ENSG00000188976__ENSMUSG00000095567  7273      4400     13774     14229    
ENSG00000187961__ENSMUSG00000078485   975       697       657       751    
ENSG00000187642__ENSMUSG00000078486    11         4        28        22    
ENSG00000187608__ENSMUSG00000035692   546       484         6         5    
ENSG00000188157__ENSMUSG00000041936 12899     10181     16978     18399    
                                    mESC_rep3
ENSG00000187634__ENSMUSG00000096351    10    
ENSG00000188976__ENSMUSG00000095567 18741    
ENSG00000187961__ENSMUSG00000078485   929    
ENSG00000187642__ENSMUSG00000078486    20    
ENSG00000187608__ENSMUSG00000035692     5    
ENSG00000188157__ENSMUSG00000041936 23923

In [18]:
orth_cols <- read.csv("../../../data/03__rna_seq/02__for_DESeq2/orths.cols.txt", sep="\t", row.names="column")
head(orth_cols)

condition sample 
hESC_rep1 hESC      sample1
hESC_rep2 hESC      sample2
mESC_rep1 mESC      sample1
mESC_rep2 mESC      sample2
mESC_rep3 mESC      sample3

## 1. expression analysis of each line separately

### human

In [40]:
dds <- DESeqDataSetFromMatrix(countData = human_cts,
                              colData = human_cols,
                              design = ~ condition)

In [42]:
dds <- estimateSizeFactors(dds)

In [43]:
human_counts <- counts(dds, normalized=TRUE)
head(human_counts)

rep1      rep2   
ENSG00000223972  1.779395  0.0000
ENSG00000227232 40.036384 40.4632
ENSG00000243485  2.669092  0.0000
ENSG00000237613  0.000000  0.0000
ENSG00000268020  0.000000  0.0000
ENSG00000240361  0.000000  0.0000

### mouse

In [44]:
dds <- DESeqDataSetFromMatrix(countData = mouse_cts,
                              colData = mouse_cols,
                              design = ~ condition)

In [45]:
dds <- estimateSizeFactors(dds)

In [46]:
mouse_counts <- counts(dds, normalized=TRUE)
head(mouse_counts)

rep1    rep2     rep3    
ENSMUSG00000102693 0.00000 0.000000 0.000000
ENSMUSG00000064842 0.00000 0.000000 0.000000
ENSMUSG00000051951 2.24968 2.140842 1.654602
ENSMUSG00000102851 0.00000 0.000000 0.000000
ENSMUSG00000103377 0.00000 0.000000 0.000000
ENSMUSG00000104017 0.00000 0.000000 0.000000

## 2. DE analysis

In [28]:
dds <- DESeqDataSetFromMatrix(countData = orth_cts,
                              colData = orth_cols,
                              design = ~ condition)
dds$condition <- factor(dds$condition, levels = c("mESC","hESC"))
dds

class: DESeqDataSet 
dim: 19858 5 
metadata(1): version
assays(1): counts
rownames(19858): ENSG00000187634__ENSMUSG00000096351
  ENSG00000188976__ENSMUSG00000095567 ...
  ENSG00000198695__ENSMUSG00000064368
  ENSG00000198727__ENSMUSG00000064370
rowData names(0):
colnames(5): hESC_rep1 hESC_rep2 mESC_rep1 mESC_rep2 mESC_rep3
colData names(2): condition sample

In [29]:
dds <- DESeq(dds)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [30]:
res <- results(dds)

In [31]:
head(res)

log2 fold change (MLE): condition hESC vs mESC 
Wald test p-value: condition hESC vs mESC 
DataFrame with 6 rows and 6 columns
                                            baseMean     log2FoldChange
                                           <numeric>          <numeric>
ENSG00000187634__ENSMUSG00000096351 36.3062016040511   3.69220902547634
ENSG00000188976__ENSMUSG00000095567 11320.6474582461  -1.32518107198941
ENSG00000187961__ENSMUSG00000078485 791.289115675006   0.20848612443869
ENSG00000187642__ENSMUSG00000078486 16.8147820929601  -1.60668673084835
ENSG00000187608__ENSMUSG00000035692 218.027475136062   6.71212814496017
ENSG00000188157__ENSMUSG00000041936 16112.5688409354 -0.660529905706031
                                                 lfcSE              stat
                                             <numeric>         <numeric>
ENSG00000187634__ENSMUSG00000096351  0.558542946013168  6.61042996215602
ENSG00000188976__ENSMUSG00000095567  0.103734993255875 -12.7746773812448
ENSG0

In [33]:
counts <- counts(dds, normalized=TRUE)
head(counts)

hESC_rep1   hESC_rep2    mESC_rep1   
ENSG00000187634__ENSMUSG00000096351    83.67935    79.231477     5.435120
ENSG00000188976__ENSMUSG00000095567  6687.91109  5203.261186 14972.669086
ENSG00000187961__ENSMUSG00000078485   896.56446   824.243874   714.174792
ENSG00000187642__ENSMUSG00000078486    10.11509     4.730237    30.436673
ENSG00000187608__ENSMUSG00000035692   502.07610   572.358731     6.522144
ENSG00000188157__ENSMUSG00000041936 11861.31791 12039.636850 18455.494101
                                    mESC_rep2    mESC_rep3   
ENSG00000187634__ENSMUSG00000096351     5.175746     8.009315
ENSG00000188976__ENSMUSG00000095567 14729.139239 15010.256691
ENSG00000187961__ENSMUSG00000078485   777.397116   744.065336
ENSG00000187642__ENSMUSG00000078486    22.773284    16.018629
ENSG00000187608__ENSMUSG00000035692     5.175746     4.004657
ENSG00000188157__ENSMUSG00000041936 19045.711775 19160.683572

## 3. write files

In [47]:
write.table(human_counts, file = "../../../data/03__rna_seq/03__diff_expr/hESC.tpm.txt", sep = "\t",
            quote = FALSE)

In [48]:
write.table(mouse_counts, file = "../../../data/03__rna_seq/03__diff_expr/mESC.tpm.txt", sep = "\t",
            quote = FALSE)

In [32]:
write.table(res, file = "../../../data/03__rna_seq/03__diff_expr/orth.DESeq2.txt", sep = "\t",
            quote = FALSE)

In [34]:
write.table(counts, file = "../../../data/03__rna_seq/03__diff_expr/orth.tpm.txt", sep = "\t",
            quote = FALSE)